# STAC Zarr 

Invoke a Common Workflow Language CommandLineTool to generate the STAC Zarr

This notebook is linked to: https://eoap.github.io/zarr-cloud-native-format/cwl-cli/stac-zarr



## Setup

In [2]:
export WORKSPACE=/workspace/zarr-cloud-native-format
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}

curl -q -L https://github.com/eoap/zarr-cloud-native-format/releases/download/0.3.0/app-water-bodies.0.3.0.cwl > ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl 2> /dev/null

## Run the STAC Zarr generation CommandLineTool

Inspect and use `cwltool` to run the STAC Zarr generation definition:


In [4]:
cat ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl | yq e '.["$graph"][7]' -

class: CommandLineTool
id: stac-zarr
requirements:
  InlineJavascriptRequirement: {}
  EnvVarRequirement:
    envDef:
      PATH: /usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
      PYTHONPATH: /app
  ResourceRequirement:
    coresMax: 1
    ramMax: 512
hints:
  DockerRequirement:
    dockerPull: ghcr.io/eoap/zarr-cloud-native-format/stac-zarr@sha256:901d20d4f27008b6109e574293a2b7699f1a59ba3ddaf43c2b01e89318b17520
baseCommand: ["python", "-m", "app"]
arguments: []
inputs:
  stac_catalog:
    type: Directory
    inputBinding:
      prefix: --stac-catalog
outputs:
  zarr_stac_catalog:
    outputBinding:
      glob: .
    type: Directory


Run the CWL description, but first prepare the parameters.

The previous step generated the water bodies detection geotif:

In [5]:
cat stac-generation-results.json 

{
    "temp_stac_catalog": {
        "location": "file:///workspace/zarr-cloud-native-format/runs/giqo2xko",
        "basename": "giqo2xko",
        "class": "Directory",
        "listing": [
            {
                "class": "File",
                "location": "file:///workspace/zarr-cloud-native-format/runs/giqo2xko/catalog.json",
                "basename": "catalog.json",
                "size": 363,
                "checksum": "sha1$60975eb0efd2cb8615bfa4b181774ea961ee7d54",
                "path": "/workspace/zarr-cloud-native-format/runs/giqo2xko/catalog.json"
            },
            {
                "class": "Directory",
                "location": "file:///workspace/zarr-cloud-native-format/runs/giqo2xko/S2A_10TFK_20210728_0_L2A",
                "basename": "S2A_10TFK_20210728_0_L2A",
                "listing": [
                    {
                        "class": "File",
                        "location": "file:///workspace/zarr-cloud-native-format/runs/giqo2xko

Let's build the job parameters file

In [6]:
cat <<EOF > stac-zarr-generation-params.yaml
stac_catalog: 
  class: Directory
  path: $( cat stac-generation-results.json | jq -r '.temp_stac_catalog.path' )
EOF

cat stac-zarr-generation-params.yaml | yq .

stac_catalog:
  class: Directory
  path: /workspace/zarr-cloud-native-format/runs/giqo2xko


In [7]:


cwltool \
    --podman \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl#stac-zarr \
    stac-zarr-generation-params.yaml > stac-zarr-generation-results.json 2> stac-zarr-generation.log

Let's look at the content of the stderr:

In [8]:
cat stac-zarr-generation.log | egrep -v "WARNING|JSHINT"

INFO /home/fbrito/.local/bin/cwltool 3.1.20250110105449
INFO Resolved '/workspace/zarr-cloud-native-format/cwl-workflow/app-water-bodies.cwl#stac-zarr' to 'file:///workspace/zarr-cloud-native-format/cwl-workflow/app-water-bodies.cwl#stac-zarr'
Error: no such object: "ghcr.io/eoap/zarr-cloud-native-format/stac-zarr@sha256:901d20d4f27008b6109e574293a2b7699f1a59ba3ddaf43c2b01e89318b17520"
INFO ['podman', 'pull', 'ghcr.io/eoap/zarr-cloud-native-format/stac-zarr@sha256:901d20d4f27008b6109e574293a2b7699f1a59ba3ddaf43c2b01e89318b17520']
Trying to pull ghcr.io/eoap/zarr-cloud-native-format/stac-zarr@sha256:901d20d4f27008b6109e574293a2b7699f1a59ba3ddaf43c2b01e89318b17520...
Getting image source signatures
Copying blob sha256:1de54e99017152f092fbe3e9f08aa9351d616f33eab466fa21bf58a11ea46c11


Copying blob sha256:cd4054fd866013877f7664cf997a9b812814d226ee5f3d830d2a275387e67f8a
Copying blob sha256:8a7a13810b7d60799d98f4a85010c5343a11ce82bf99a2a9231356ee56cd32f9
Copying blob sha256:a2318d6c47ec9cac5acc500c47c79602bcf953cec711a18bc898911a0984365b
Copying blob sha256:3a69aa919be22561b20b4697f9879ce22216926b78dd287ab97cbbb17ecb3792
Copying blob sha256:ddee35dfe011992ed9d752dc61e7dc4377fda83f63e03c0e193a4a4e6714cc2b
Copying config sha256:09de2403c285fded0ba732081f5f125e9e19c815d20db91d1147a4b7e078c826
Writing manifest to image destination
09de2403c285fded0ba732081f5f125e9e19c815d20db91d1147a4b7e078c826
INFO [job stac-zarr] /tmp/3qsjkdkx$ podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/tmp/3qsjkdkx,target=/sMzpvK \
    --mount=type=bind,source=/tmp/ow5up7t4,target=/tmp \
    --mount=type=bind,source=/workspace/zarr-cloud-native-format/runs/giqo2xko,target=/var/lib/cwl/stg69cdcb7d-820e-4e48-abdf-641d0c22b48e/giqo2xko,readonly \
    --workdir=/sMzpvK 

Let's inspect the stdout produced. 

In [9]:
cat stac-zarr-generation-results.json | jq . -

{
  "zarr_stac_catalog": {
    "location": "file:///workspace/zarr-cloud-native-format/runs/3qsjkdkx",
    "basename": "3qsjkdkx",
    "class": "Directory",
    "listing": [
      {
        "class": "Directory",
        "location": "file:///workspace/zarr-cloud-native-format/runs/3qsjkdkx/water-bodies",
        "basename": "water-bodies",
        "listing": [
          {
            "class": "Directory",
            "location": "file:///workspace/zarr-cloud-native-format/runs/3qsjkdkx/water-bodies/result.zarr",
            "basename": "result.zarr",
            "listing": [
              {
                "class": "File",
                "location": "file:///workspace/zarr-cloud-native-format/runs/3qsjkdkx/water-bodies/result.zarr/.zgroup",
                "basename": ".zgroup",
                "size": 24,
                "checksum": "sha1$63b0fcd7748c79d0de97705fb1b8ed5fcc5ac788",
                "path": "/workspace/zarr-cloud-native-format/runs/3qsjkdkx/water-bodies/result.zarr/.zgro

              {
                "class": "Directory",
                "location": "file:///workspace/zarr-cloud-native-format/runs/3qsjkdkx/water-bodies/result.zarr/data",
                "basename": "data",
                "listing": [
                  {
                    "class": "File",
                    "location": "file:///workspace/zarr-cloud-native-format/runs/3qsjkdkx/water-bodies/result.zarr/data/0.3.3",
                    "basename": "0.3.3",
                    "size": 30018,
                    "checksum": "sha1$f4aef2e15130a168fbfc8d0016fcd234d7e65811",
                    "path": "/workspace/zarr-cloud-native-format/runs/3qsjkdkx/water-bodies/result.zarr/data/0.3.3"
                  },
                  {
                    "class": "File",
                    "location": "file:///workspace/zarr-cloud-native-format/runs/3qsjkdkx/water-bodies/result.zarr/data/0.4.8",
                    "basename": "0.4.8",
                    "size": 1336,
                    "che